In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime, timedelta



In [7]:
def parse_data(inp_df:pd.DataFrame):
    df_to_parse = inp_df.copy().reset_index(drop=True)
    # Handle NaN values in json
    df_to_parse["json_column"] = df_to_parse["contracts"].apply(lambda x: json.loads(x) if pd.notna(x) else [])
    # Handle single dict values in json
    df_to_parse["json_column"] = df_to_parse['json_column'].apply(lambda x: x if type(x) in [list] else [x])
    df_exploded = df_to_parse.explode('json_column').reset_index(drop=True)
    df_exploded['json_column']=df_exploded['json_column'].apply(lambda x: x if pd.notna(x)  else {})
    normalized_df = pd.json_normalize(df_exploded['json_column']).reset_index(drop=True)
    parsed_df = df_exploded.reset_index(drop=True).drop('json_column', axis=1).join(normalized_df)
    return parsed_df.reset_index(drop=True)

def build_feature_claim_cnt(bla: pd.DataFrame, n_days: int):
    inp_df = bla.copy()
    # Get the current date
    current_date = datetime.now()

    # Calculate cutoff date
    lower_bound = current_date - timedelta(days=n_days)
    
    # Feature name based on cutoff date
    feature_name = 'tot_claim_cnt_l'+str(n_days)+'d'
    
    # Count claims for last n days. ignoring older claims and rows with missing claim_date 
    inp_df["claim_meta"]=np.where((inp_df["claim_date"].isna())|(pd.to_datetime(inp_df["claim_date"])<lower_bound), np.nan,inp_df["claim_id"] )
    claim_counts =inp_df.groupby('id')['claim_meta'].apply(lambda x: x.notna().sum()).reset_index()

    claim_counts.columns = ['id', 'tot_claim_cnt_l'+str(n_days)+'d']
    df_res = inp_df.merge(claim_counts, on='id', how='left')

    #if no claims during last n days, set value -3
    df_res[feature_name]=np.where(df_res[feature_name]==0, -3,df_res[feature_name])
    
    return df_res
    
def calculate_loan_counts(inp_df: pd.DataFrame):
    # Count all loans by id
    inp_df["loan_summa"]=pd.to_numeric(inp_df["loan_summa"], errors='coerce')
    loan_counts =inp_df.groupby('id')['loan_summa'].apply(lambda x: (x>=0).sum()).reset_index()
    loan_counts.columns = ['id', 'tot_loan_cnt']
    return inp_df.merge(loan_counts, on='id', how='left')
    
def calculate_claim_counts(inp_df: pd.DataFrame):
    # Count claims by id
    claim_counts =inp_df.groupby('id')['claim_id'].apply(lambda x: x.notna().sum()).reset_index()
    claim_counts.columns = ['id', 'tot_claim_cnt']
    return inp_df.merge(claim_counts, on='id', how='left')

def build_feature_disb_bank_loan_wo_tbc(inp_df: pd.DataFrame):
    # calculate sum of exposure of loans ignoring 
    inp_df["loan_meta"] = np.where((~inp_df["bank"].isin(['LIZ', 'LOM', 'MKO', 'SUG', np.nan])) &(~inp_df["contract_date"].isna()), inp_df["loan_summa"],0)
    loans_sum =inp_df.groupby('id')['loan_meta'].apply(lambda x: int(x.sum())).reset_index()
    loans_sum.columns = ['id', 'disb_bank_loan_wo_tbc']
    res_df= inp_df.merge(loans_sum, on='id', how='left')
    
    #if no previous claims, set value -3, if no previous loan, set value -1
    res_df['disb_bank_loan_wo_tbc']=np.where(res_df['tot_claim_cnt']<=0, -3,np.where(res_df['tot_loan_cnt']<=0,-1,res_df['disb_bank_loan_wo_tbc']))
    return res_df
    
def build_feature_day_sinlastloan(inp_df: pd.DataFrame):
    # format date with %Y-%M-%D
    inp_df["last_loan_date_meta"]=pd.to_datetime(inp_df["contract_date"].apply(lambda x:str(x)[-4:]+"-"+str(x)[3:5]+"-"+str(x)[:2] if pd.notna(x)else np.nan),errors='coerce')
   
    # calculate days diff between existing previous loan date and current loan application date. 
    last_loans =inp_df.groupby('id')['last_loan_date_meta'].apply(lambda x: x.max()).reset_index()
    last_loans.columns=["id", "last_loan_date"]
    res_df = inp_df.merge(last_loans, on='id', how='left')
    res_df["day_sinlastloan"] = pd.to_datetime(res_df["application_date"]) - res_df["last_loan_date"]
    res_df["day_sinlastloan"]=(res_df["day_sinlastloan"]/ pd.Timedelta(days=1)).fillna(-1).astype(int)  
    
    #if no previous claims, set value -3, if no previous loan, set value -1
    res_df['day_sinlastloan']=np.where(res_df['day_sinlastloan']>-1, res_df['day_sinlastloan'],np.where(res_df['tot_claim_cnt']<=0,-3,-1))
    return res_df

In [8]:
# Read data from file 
loan_applications = pd.read_csv("data.csv")
loan_applications["application_date"]=pd.to_datetime(loan_applications["application_date"]).apply(lambda x: x.strftime('%Y-%m-%d'))
loan_applications["id"]=loan_applications["id"].astype("int64")

In [9]:
# Run predefined functions to parse json and deliver requested features
n_days = 180
feature_name_1 = 'tot_claim_cnt_l'+str(n_days)+'d'

# Parsing step
parsed_json = parse_data(loan_applications)

# Build feature 1
res = build_feature_claim_cnt(parsed_json, n_days)

# Some pre-calculations for filling missing features' values
res = calculate_loan_counts(res)
res = calculate_claim_counts(res)

# Build feature 2
res = build_feature_disb_bank_loan_wo_tbc(res)

# Building feature 3
res = build_feature_day_sinlastloan(res)
columns_list = list(loan_applications.columns)+[feature_name_1,"disb_bank_loan_wo_tbc","day_sinlastloan"]

# Deduplicate exploaded rows
res = res[columns_list].drop_duplicates().reset_index(drop=True)

# Save result as csv
res.to_csv("contract_features.csv")

In [10]:
res.head(20)

,id,application_date,contracts,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan
0,2925210,2024-02-12,NaN,-3,-3,-3
1,2925211,2024-02-12,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",15,0,427
2,2925212,2024-02-12,NaN,-3,-3,-3
3,2925213,2024-02-12,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",15,0,427
4,2925214,2024-02-12,NaN,-3,-3,-3
5,2925215,2024-02-12,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",15,0,427
6,2925216,2024-02-12,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",15,0,427
7,2925217,2024-02-12,NaN,-3,-3,-3
8,2925218,2024-02-12,"[{""contract_id"": """", ""bank"": ""062"", ""summa"": ""...",11,0,288
9,2925219,2024-02-12,NaN,-3,-3,-3
